In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU, ELU, BatchNormalization, Dropout
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from stargazer.stargazer import Stargazer
import seaborn as sns
import matplotlib.pyplot as plt
from keras.utils import plot_model

In [2]:
from keras.models import load_model

In [3]:
model = load_model('heston_model.h5')

2023-04-16 15:14:53.914197: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-16 15:14:53.914227: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [4]:
df1 = pd.read_csv("HestonSimulation.csv", index_col = 0)
df2 = pd.read_csv("HestonSimulation2.csv", index_col = 0)
data = df1.append(df2)
Y = data["call_price"]
X = data.drop(columns=["call_price"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

/var/folders/jv/604w31hs1v964hc3p1jvb1c40000gn/T/ipykernel_36687/1280459780.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df1.append(df2)


In [11]:
import time 
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print(end - start)

63/63 [==============================] - 0s 7ms/step
0.51375412940979


In [6]:
pd.DataFrame(y_pred)

,0
0,67.691696
1,38.463688
2,143.469162
3,4.221910
4,78.216385
...,...
1995,171.611542
1996,88.209839
1997,3.423959
1998,25.721973


In [7]:
pd.DataFrame(y_test)

,call_price
650,62.234715
2041,40.619595
8668,157.760566
1114,3.910600
3902,84.255233
...,...
5022,187.209878
13,104.486133
3927,3.144596
9560,26.097666


In [8]:
mape = np.abs(y_test.values.reshape(len(y_test), 1) - y_pred)/y_test.values.reshape(len(y_test), 1)

In [9]:
subData = X_test.copy()
subData["mape"] = mape
(subData).describe()

,S,K,T,r,v_0,theta,kappa,rho,xi,mape
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,256.457002,258.298689,1.589205,0.055064,0.471561,0.412250,4.941768,0.003857,0.902435,0.090583
std,142.664805,148.360080,0.840519,0.026297,0.248322,0.232362,2.892067,0.578870,0.749649,0.116981
min,10.000000,8.528634,0.083333,0.010000,0.050000,0.010000,0.000000,-0.990000,0.000000,0.000023
25%,132.500000,127.679087,0.888409,0.031818,0.256061,0.209495,2.424242,-0.510856,0.302249,0.032044
50%,258.433433,255.476395,1.582541,0.054545,0.470707,0.416970,4.848485,0.019820,0.700075,0.066407
75%,381.056056,381.151776,2.334334,0.078182,0.685354,0.616465,7.373737,0.502432,1.357865,0.113322
max,500.000000,671.148889,3.000000,0.100000,0.900000,0.800000,10.000000,0.990000,3.789454,2.951419
